In [0]:
use edsm.gold;

create
or replace view v_powerPlay as
select
  valid_from,
  case
    when valid_to is null then to_date("2099-12-31", "yyyy-MM-dd")
    else valid_to
  end as valid_to,
  *
except
  (valid_from, valid_to)
from
  (
    select
      to_date(valid_from) as valid_from,
      to_date(valid_to) as valid_to,
      *
    except
      (valid_from, valid_to)
    from
      (
        select
          created_on as valid_from,
          case
            when deleted_on is null then lead(created_on) OVER (PARTITION BY name, id, id64, power ORDER BY created_on)
            when deleted_on is not null then to_date(deleted_on)
            else null
          end as valid_to,
          current_flag,
          *
        except
          (created_on, current_flag)
        from
          powerplay
      )
  );